In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
print(torch.__version__)

0.4.0


In [0]:
import pandas as pd
import torch

df = pd.read_csv('SUNTV.csv')
df = df.drop(['Date Time'], 1)


#####################################################################

def classification_data(df, interval=10):
  c=[]
  for i, row in df.iterrows():
    try:
      if df.iloc[i]['Close'] < df.iloc[i+interval]['Close']:
        c.append(1)
      else:
        c.append(0)
    except: 
      try:
        df =df.drop(df.index[[i]])
      except:
        continue

  df = df.iloc[:len(c)]
  se = pd.Series(c)
  df['class'] = se.values
  df.tail()
  return df

df = classification_data(df)

###################################################################


y = df['class'].values
X = df.drop('class', 1).values

from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)

########################################################################
import numpy as np

X_n = []
y_n = []

for i in range(len(x_scaled) - 1000):
    X_n.append(x_scaled[i:i+1000].T)
    y_n.append(np.array(y[i:i+1000]))
    
##########################################################################3
X_train = X_n[:int(len(X_n)*0.8)]
X_test = X_n[len(X_n[:int(len(X_n)*0.8)]):]

y_train = y_n[:int(len(y_n)*0.8)]
y_test = y_n[len(y_n[:int(len(y_n)*0.8)]):]

import torch.utils.data as utils
tensor_x_tr = torch.stack([torch.Tensor(i) for i in X_n]) 
tensor_y_tr = torch.stack([torch.Tensor(i) for i in y_n])
my_dataset_tr = utils.TensorDataset(tensor_x_tr,tensor_y_tr) 
trainloader = utils.DataLoader(my_dataset_tr, batch_size=1024) 

###

tensor_x_te = torch.stack([torch.Tensor(i) for i in X_test]) 
tensor_y_te = torch.stack([torch.Tensor(i) for i in y_test])
my_dataset_te = utils.TensorDataset(tensor_x_te,tensor_y_te) 
testloader = utils.DataLoader(my_dataset_te, batch_size=1024) 

##############################################################################

import torch.nn as nn
import torch.nn.functional as F


class ConvModel(nn.Module):
  def __init__(self, input_channel=5, kernel_size=10):
    super(ConvModel, self).__init__()
    self.c1 = nn.Conv1d(input_channel, 32, kernel_size)
    self.c2 = nn.Conv1d(32, 64, kernel_size)
    self.c3 = nn.Conv1d(64, 128, kernel_size)
    self.c4 = nn.Conv1d(128, 256, kernel_size)
    self.c5 = nn.Conv1d(256, 256, kernel_size)
    self.c6 = nn.Conv1d(256, 256, kernel_size)
    self.mp = nn.MaxPool1d(2)
    self.relu = nn.ReLU()
    self.lrelu = nn.LeakyReLU(0.01)
    self.dp = nn.Dropout(0.8)
    self.fc1 = nn.Linear(3328, 2000)
    self.fc2 = nn.Linear(2000, 1000)
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, x):
    x = self.c1(x)
    x = self.relu(x)

    x = self.mp(x)
    x = self.c2(x)
    x = self.lrelu(x)
    x = self.mp(x)
    
    
    x = self.c3(x)
    x = self.lrelu(x)
    x = self.mp(x)
    x = self.c4(x)
    x = self.lrelu(x)
    x = self.mp(x)
    x = self.c5(x)
    x = self.lrelu(x)
    x = self.mp(x)
    x = self.c6(x)
    x = self.lrelu(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    x = self.dp(x)
    x = self.fc2(x)
#     x = self.sigmoid(x)
    x = F.log_softmax(x, dim=1)
    return x
    
    
    ########################################################################33
    
model =  ConvModel(5)

criterion = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

##################################################################################33

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

epochs = 15
steps = 0
running_loss = 0
print_every = 1
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
#         inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        
        optimizer.zero_grad()
        print(f'labels s={labels.view(-1).shape}, data s={inputs.shape}')
        logps = model(inputs)
        print(f'logps shape{logps.view(-1).shape}')
        loss = criterion(logps.view(1,-1), labels.view(1,-1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
#         if steps % print_every == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
    #                     inputs, labels = inputs.to(device), labels.to(device).long()
                labels=labels.long()
#                 print(f'labels shape = {labels.shape} ')
                print(f'labels s={labels.shape}, data s={inputs.shape}')
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)

                test_loss += batch_loss.item()

                # Calculate accuracy
                ps = torch.exp(logps)
                print(f'epoch={epoch} ps shape = {ps.shape}')
                print(f'labels shape = {labels.shape} ')
#                 top_p, top_class = ps.topk(1, dim=1)
#                 print('topp shape= ',top_p.shape)
                equals = ps == labels
#   .view(*ps.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    #                     print('acc=',accuracy)

#         print(f"Epoch {epoch+1}/{epochs}.. "
#               f"Train loss: {running_loss/print_every}.. "
#               f"Test loss: {test_loss/len(testloader)}.. "
#               f"Test accuracy: {accuracy/len(testloader)}")
        running_loss = 0
        model.train()

labels s=torch.Size([1024000]), data s=torch.Size([1024, 5, 1000])


In [0]:
import pandas as pd

df = pd.read_csv('SUNTV.csv')
df = df.drop(['Date Time'], 1)
df.head(20)

,Close,High,Low,Open,Volume
0,411.15,413.00,404.10,409.00,176240
1,414.05,414.70,410.00,411.15,75844
2,410.20,414.40,410.00,414.40,40789
3,410.25,412.50,410.00,410.20,17987
4,410.00,410.35,408.80,410.35,57179
5,410.25,411.25,408.50,410.00,81484
6,409.00,410.25,409.00,410.00,46254
7,415.75,419.40,410.00,410.00,171503
8,415.75,416.25,413.20,415.75,116767
9,414.15,416.90,411.75,415.75,78892


In [0]:
def classification_data(df, interval=10):
  c=[]
  for i, row in df.iterrows():
    try:
      if df.iloc[i]['Close'] < df.iloc[i+interval]['Close']:
        c.append(1)
      else:
        c.append(0)
    except: 
      try:
        df =df.drop(df.index[[i]])
      except:
        continue

  df = df.iloc[:len(c)]
  se = pd.Series(c)
  df['class'] = se.values
  df.tail()
  return df

df = classification_data(df)

In [0]:
df.head(20)

,Close,High,Low,Open,Volume,class
0,411.15,413.00,404.10,409.00,176240,1
1,414.05,414.70,410.00,411.15,75844,1
2,410.20,414.40,410.00,414.40,40789,1
3,410.25,412.50,410.00,410.20,17987,1
4,410.00,410.35,408.80,410.35,57179,1
5,410.25,411.25,408.50,410.00,81484,1
6,409.00,410.25,409.00,410.00,46254,1
7,415.75,419.40,410.00,410.00,171503,0
8,415.75,416.25,413.20,415.75,116767,0
9,414.15,416.90,411.75,415.75,78892,0


In [0]:
len(df)

6176

In [0]:
y = df['class'].values
y[:10]

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [0]:
X = df.drop('class', 1).values
X[:3]

array([[   411.15,    413.  ,    404.1 ,    409.  , 176240.  ],
       [   414.05,    414.7 ,    410.  ,    411.15,  75844.  ],
       [   410.2 ,    414.4 ,    410.  ,    414.4 ,  40789.  ]])

In [0]:
from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
x_scaled[:3]

array([[0.18402965, 0.1838875 , 0.17836732, 0.18043088, 0.0146644 ],
       [0.18749626, 0.18591348, 0.18546592, 0.18300419, 0.00597011],
       [0.18289403, 0.18555595, 0.18546592, 0.18689408, 0.00293435]])

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class ConvModel(nn.Module):
  def __init__(self, input_channel=5, kernel_size=10):
    super(ConvModel, self).__init__()
    self.c1 = nn.Conv1d(input_channel, 32, kernel_size)
    self.c2 = nn.Conv1d(32, 64, kernel_size)
    self.c3 = nn.Conv1d(64, 128, kernel_size)
    self.c4 = nn.Conv1d(128, 256, kernel_size)
    self.c5 = nn.Conv1d(256, 256, kernel_size)
    self.c6 = nn.Conv1d(256, 256, kernel_size)
    self.mp = nn.MaxPool1d(2)
    self.relu = nn.ReLU()
    self.lrelu = nn.LeakyReLU(0.01)
    self.dp = nn.Dropout(0.8)
    self.fc1 = nn.Linear(3328, 2000)
    self.fc2 = nn.Linear(2000, 1000)
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, x):
    x = self.c1(x)
    x = self.relu(x)

    x = self.mp(x)
    x = self.c2(x)
    x = self.lrelu(x)
    x = self.mp(x)
    
    
    x = self.c3(x)
    x = self.lrelu(x)
    x = self.mp(x)
    x = self.c4(x)
    x = self.lrelu(x)
    x = self.mp(x)
    x = self.c5(x)
    x = self.lrelu(x)
    x = self.mp(x)
    x = self.c6(x)
    x = self.lrelu(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    x = self.dp(x)
    x = self.fc2(x)
#     x = self.sigmoid(x)
    x = F.log_softmax(x, dim=1)
    return x
    
 

In [0]:
model = ConvModel(5)

import torch
input1 = torch.randn(1, 5, 1000)
model(input1)

tensor([[-6.8772, -6.8785, -6.8782, -6.9056, -6.9166, -6.9056, -6.8815,
         -6.9153, -6.9097, -6.9340, -6.9342, -6.9256, -6.8809, -6.8585,
         -6.9092, -6.9058, -6.9096, -6.9326, -6.9200, -6.9568, -6.9080,
         -6.9066, -6.8840, -6.8989, -6.8950, -6.9111, -6.8817, -6.8984,
         -6.8870, -6.9188, -6.8935, -6.8936, -6.8937, -6.9358, -6.9094,
         -6.9418, -6.9161, -6.8832, -6.8708, -6.8898, -6.9276, -6.8690,
         -6.9449, -6.9177, -6.8858, -6.9207, -6.9490, -6.8968, -6.8714,
         -6.9144, -6.8904, -6.8971, -6.9158, -6.9047, -6.8909, -6.9225,
         -6.9077, -6.9059, -6.8646, -6.9075, -6.9117, -6.9389, -6.9093,
         -6.8965, -6.8860, -6.8854, -6.9000, -6.9076, -6.9308, -6.8818,
         -6.9001, -6.8914, -6.9357, -6.9260, -6.8893, -6.8983, -6.9145,
         -6.9307, -6.9063, -6.9436, -6.9054, -6.9431, -6.9094, -6.8612,
         -6.9234, -6.9191, -6.8917, -6.9229, -6.8749, -6.9166, -6.8710,
         -6.9108, -6.9010, -6.9325, -6.8865, -6.9194, -6.9228, -

In [0]:
import numpy as np

X_n = []
y_n = []

for i in range(len(x_scaled) - 1000):
    X_n.append(x_scaled[i:i+1000].T)
    y_n.append(np.array(y[i:i+1000]))


In [0]:
X_n[0].shape

(5, 1000)

In [0]:
X_train = X_n[:int(len(X_n)*0.8)]
X_test = X_n[len(X_n[:int(len(X_n)*0.8)]):]

y_train = y_n[:int(len(y_n)*0.8)]
y_test = y_n[len(y_n[:int(len(y_n)*0.8)]):]

In [0]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=0.2, random_state=42)



In [0]:
import torch.utils.data as utils

tensor_x_tr = torch.stack([torch.Tensor(i) for i in X_n]) 
tensor_y_tr = torch.stack([torch.Tensor(i) for i in y_n])

my_dataset_tr = utils.TensorDataset(tensor_x_tr,tensor_y_tr) 
trainloader = utils.DataLoader(my_dataset_tr, batch_size=1024) 






tensor_x_te = torch.stack([torch.Tensor(i) for i in X_test]) 
tensor_y_te = torch.stack([torch.Tensor(i) for i in y_test])

my_dataset_te = utils.TensorDataset(tensor_x_te,tensor_y_te) 
testloader = utils.DataLoader(my_dataset_te, batch_size=1024) 

In [0]:
for data, target in trainloader:
    print(data.shape)
    print(target.shape)
    break

torch.Size([1024, 5, 1000])
torch.Size([1024, 1000])


In [0]:
model =  ConvModel(5)
print(model)

ConvModel(
  (c1): Conv1d(5, 32, kernel_size=(10,), stride=(1,))
  (c2): Conv1d(32, 64, kernel_size=(10,), stride=(1,))
  (c3): Conv1d(64, 128, kernel_size=(10,), stride=(1,))
  (c4): Conv1d(128, 256, kernel_size=(10,), stride=(1,))
  (c5): Conv1d(256, 256, kernel_size=(10,), stride=(1,))
  (c6): Conv1d(256, 256, kernel_size=(10,), stride=(1,))
  (mp): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (lrelu): LeakyReLU(negative_slope=0.01)
  (dp): Dropout(p=0.8)
  (fc1): Linear(in_features=3328, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=1000, bias=True)
  (sigmoid): Sigmoid()
)


In [0]:
model =  ConvModel(5)

criterion = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [0]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device);

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

epochs = 5
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
#         inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        
        optimizer.zero_grad()
        
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
#         if steps % print_every == 0:
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device), labels.to(device).long()
            labels=labels.long()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#                     print('acc=',accuracy)

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/print_every}.. "
          f"Test loss: {test_loss/len(testloader)}.. "
          f"Test accuracy: {accuracy/len(testloader)}")
    running_loss = 0
    model.train()

Epoch 1/5.. Train loss: 8.277824211120606.. Test loss: 6.879637718200684.. Test accuracy: 0.0
Epoch 2/5.. Train loss: 8.248946952819825.. Test loss: 6.8454749584198.. Test accuracy: 0.73876953125
Epoch 3/5.. Train loss: 8.20350341796875.. Test loss: 6.801614284515381.. Test accuracy: 0.73876953125
Epoch 4/5.. Train loss: 8.149008750915527.. Test loss: 6.752331733703613.. Test accuracy: 0.73876953125
Epoch 5/5.. Train loss: 8.089841842651367.. Test loss: 6.699279546737671.. Test accuracy: 0.73876953125


In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

epochs = 15
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
#         inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        
        optimizer.zero_grad()
        
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
#         if steps % print_every == 0:
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device), labels.to(device).long()
            labels=labels.long()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#                     print('acc=',accuracy)

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/print_every}.. "
          f"Test loss: {test_loss/len(testloader)}.. "
          f"Test accuracy: {accuracy/len(testloader)}")
    running_loss = 0
    model.train()

Epoch 1/15.. Train loss: 8.047703742980957.. Test loss: 6.682156085968018.. Test accuracy: 0.73876953125
Epoch 2/15.. Train loss: 8.015548419952392.. Test loss: 6.644542455673218.. Test accuracy: 0.73876953125
Epoch 3/15.. Train loss: 7.965180778503418.. Test loss: 6.59398889541626.. Test accuracy: 0.73876953125
Epoch 4/15.. Train loss: 7.9010422706604.. Test loss: 6.533164978027344.. Test accuracy: 0.73876953125
Epoch 5/15.. Train loss: 7.825938987731933.. Test loss: 6.461606979370117.. Test accuracy: 0.73876953125
Epoch 6/15.. Train loss: 7.738792133331299.. Test loss: 6.375495433807373.. Test accuracy: 0.73876953125
Epoch 7/15.. Train loss: 7.632960510253906.. Test loss: 6.26566481590271.. Test accuracy: 0.73876953125
Epoch 8/15.. Train loss: 7.494869804382324.. Test loss: 6.108920097351074.. Test accuracy: 0.73876953125
Epoch 9/15.. Train loss: 7.289493370056152.. Test loss: 5.834909915924072.. Test accuracy: 0.73876953125
Epoch 10/15.. Train loss: 6.884248161315918.. Test loss: 5.

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

epochs = 15
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
#         inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        
        optimizer.zero_grad()
        
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
#         if steps % print_every == 0:
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device), labels.to(device).long()
            labels=labels.long()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#                     print('acc=',accuracy)

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/print_every}.. "
          f"Test loss: {test_loss/len(testloader)}.. "
          f"Test accuracy: {accuracy/len(testloader)}")
    running_loss = 0
    model.train()

Epoch 1/15.. Train loss: 3.1828015804290772.. Test loss: 3.186503291130066.. Test accuracy: 0.26123046875
Epoch 2/15.. Train loss: 3.0239305019378664.. Test loss: 3.1701241731643677.. Test accuracy: 0.26123046875
Epoch 3/15.. Train loss: 2.866197347640991.. Test loss: 3.172333598136902.. Test accuracy: 0.26123046875
Epoch 4/15.. Train loss: 2.7391216278076174.. Test loss: 3.099419951438904.. Test accuracy: 0.26123046875
Epoch 5/15.. Train loss: 2.5808584213256838.. Test loss: 2.9255017042160034.. Test accuracy: 0.26123046875
Epoch 6/15.. Train loss: 2.421225643157959.. Test loss: 2.692746937274933.. Test accuracy: 0.26123046875
Epoch 7/15.. Train loss: 2.2383219957351685.. Test loss: 2.448759377002716.. Test accuracy: 0.26123046875
Epoch 8/15.. Train loss: 2.084571385383606.. Test loss: 2.215104103088379.. Test accuracy: 0.26123046875
Epoch 9/15.. Train loss: 1.917207670211792.. Test loss: 1.9873838424682617.. Test accuracy: 0.26123046875
Epoch 10/15.. Train loss: 1.7300297975540162.. 

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

epochs = 15
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
#         inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        
        optimizer.zero_grad()
        
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
#         if steps % print_every == 0:
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device), labels.to(device).long()
            labels=labels.long()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#                     print('acc=',accuracy)

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/print_every}.. "
          f"Test loss: {test_loss/len(testloader)}.. "
          f"Test accuracy: {accuracy/len(testloader)}")
    running_loss = 0
    model.train()

Epoch 1/15.. Train loss: 0.960979413986206.. Test loss: 0.8127917349338531.. Test accuracy: 0.26123046875
Epoch 2/15.. Train loss: 0.940903389453888.. Test loss: 0.7788000404834747.. Test accuracy: 0.26123046875
Epoch 3/15.. Train loss: 0.9320174217224121.. Test loss: 0.7417958974838257.. Test accuracy: 0.26123046875
Epoch 4/15.. Train loss: 0.8921349048614502.. Test loss: 0.710993081331253.. Test accuracy: 0.26123046875
Epoch 5/15.. Train loss: 0.8851672410964966.. Test loss: 0.6896490454673767.. Test accuracy: 0.73876953125
Epoch 6/15.. Train loss: 0.8996837973594666.. Test loss: 0.6781420409679413.. Test accuracy: 0.73876953125
Epoch 7/15.. Train loss: 0.8852800846099853.. Test loss: 0.6732870936393738.. Test accuracy: 0.73876953125
Epoch 8/15.. Train loss: 0.8686652064323426.. Test loss: 0.6729720830917358.. Test accuracy: 0.73876953125
Epoch 9/15.. Train loss: 0.8810382843017578.. Test loss: 0.6764772236347198.. Test accuracy: 0.73876953125
Epoch 10/15.. Train loss: 0.883145010471

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

epochs = 15
steps = 0
running_loss = 0
print_every = 1
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
#         inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()
        
        optimizer.zero_grad()
        print(f'labels s={labels.view(-1).shape}, data s={data.shape}')
        logps = model(inputs)
        print(f'logps shape{logps.view(-1).shape}')
        loss = criterion(logps.view(1,-1), labels.view(1,-1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
#         if steps % print_every == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
    #                     inputs, labels = inputs.to(device), labels.to(device).long()
                labels=labels.long()
#                 print(f'labels shape = {labels.shape} ')
                print(f'labels s={labels.shape}, data s={data.shape}')
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)

                test_loss += batch_loss.item()

                # Calculate accuracy
                ps = torch.exp(logps)
                print(f'epoch={epoch} ps shape = {ps.shape}')
                print(f'labels shape = {labels.shape} ')
#                 top_p, top_class = ps.topk(1, dim=1)
#                 print('topp shape= ',top_p.shape)
                equals = ps == labels
#   .view(*ps.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    #                     print('acc=',accuracy)

#         print(f"Epoch {epoch+1}/{epochs}.. "
#               f"Train loss: {running_loss/print_every}.. "
#               f"Test loss: {test_loss/len(testloader)}.. "
#               f"Test accuracy: {accuracy/len(testloader)}")
        running_loss = 0
        model.train()

labels s=torch.Size([1024000]), data s=torch.Size([1024, 5, 1000])
